# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import time

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,wailua homesteads,22.0669,-159.3780,28.44,75,0,7.72,US,1690490038
1,1,alofi,-19.0595,-169.9187,20.94,88,100,1.54,NU,1690490038
2,2,grytviken,-54.2811,-36.5092,-0.89,97,100,3.36,GS,1690490038
3,3,zarzis,33.5040,11.1122,26.73,70,0,5.70,TN,1690490038
4,4,labytnangi,66.6572,66.4183,15.63,92,78,3.03,RU,1690490038


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Create a date variable to display date in map titles
# convert from unix timestamp
_ = time.localtime(city_data_df["Date"][1])
date = f"{_.tm_year}-{_.tm_mon}-{_.tm_mday}"

In [4]:
%%capture --no-display

# Configure the map plot
map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    color="City",
    size="Humidity",
    frame_width=800,
    frame_height=600,
    title=f'City Data: Humidity and Location on {date}',
    fontsize={'title': '16pt'},
    line_color='black'
)

# Display the map
map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
df = city_data_df[(city_data_df["Max Temp"] > 25) & (city_data_df["Humidity"] > 50) & (city_data_df["Cloudiness"] < 30)]
# Note, this is hardcoded to my supposed ideal weather conditions
# Someday, I may make this dynamic to take in anybody's ideal weather conditions

# Drop any rows with null values
df = df.dropna(how="any")

# Display sample data
df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,wailua homesteads,22.0669,-159.3780,28.44,75,0,7.72,US,1690490038
3,3,zarzis,33.5040,11.1122,26.73,70,0,5.70,TN,1690490038
8,8,bethel,41.3712,-73.4140,33.34,64,20,6.69,US,1690490039
27,27,hillsborough,37.5741,-122.3794,29.13,66,20,7.72,US,1690490040
46,46,kuching,1.5500,110.3333,26.10,90,20,2.24,MY,1690490026


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,wailua homesteads,US,22.0669,-159.3780,75,
3,zarzis,TN,33.5040,11.1122,70,
8,bethel,US,41.3712,-73.4140,64,
27,hillsborough,US,37.5741,-122.3794,66,
46,kuching,MY,1.5500,110.3333,90,
61,princeville,CA,45.7668,-61.2985,72,
67,touros,BR,-5.1989,-35.4608,83,
72,sulak,RU,43.2734,47.5146,78,
108,misratah,LY,32.3754,15.0925,66,
112,georgetown,MY,5.4112,100.3354,92,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey" : geoapify_key,
    "categories" : "accommodation.hotel",
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    # Convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
zarzis - nearest hotel: Zephir Hotel & Spa
bethel - nearest hotel: Hampton Inn Danbury
hillsborough - nearest hotel: Crowne Plaza San Francisco Airport
kuching - nearest hotel: Hotel Samudera
princeville - nearest hotel: No hotel found
touros - nearest hotel: Pousada Atlântico
sulak - nearest hotel: No hotel found
misratah - nearest hotel: الفندق السياحى
georgetown - nearest hotel: Page 63 hostel
dickson - nearest hotel: No hotel found
gaspe - nearest hotel: Hotel Plante
papao - nearest hotel: Hiti Moana Villa Lodge
lihue - nearest hotel: Kauai Palms
uturoa - nearest hotel: Hawaiki Nui hotel
atafu village - nearest hotel: No hotel found
hasaki - nearest hotel: 大新旅館
ixtapa - nearest hotel: Ma. Cristina
kapa'a - nearest hotel: Pono Kai Resort
barra de santo antonio - nearest hotel: Resort Fazenda Fiore
rivesaltes - nearest hotel: Kyriad
tomah - nearest hotel: No hotel found
aroma - nearest hotel: 

,City,Country,Lat,Lng,Humidity,Hotel Name
0,wailua homesteads,US,22.0669,-159.3780,75,Hilton Garden Inn Kauai Wailua Bay
3,zarzis,TN,33.5040,11.1122,70,Zephir Hotel & Spa
8,bethel,US,41.3712,-73.4140,64,Hampton Inn Danbury
27,hillsborough,US,37.5741,-122.3794,66,Crowne Plaza San Francisco Airport
46,kuching,MY,1.5500,110.3333,90,Hotel Samudera
61,princeville,CA,45.7668,-61.2985,72,No hotel found
67,touros,BR,-5.1989,-35.4608,83,Pousada Atlântico
72,sulak,RU,43.2734,47.5146,78,No hotel found
108,misratah,LY,32.3754,15.0925,66,الفندق السياحى
112,georgetown,MY,5.4112,100.3354,92,Page 63 hostel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    color="City",
    line_color="black",
    size=80,
    frame_width=800,
    frame_height=600,
    hover_cols=["Hotel Name", "Country", "Humidity"],
    title=f'Possible Vacation Cities: Humidity and Nearest Hotel within 10km on {date}',
    fontsize={'title': '16pt'},
)

# Display the map
map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country,Humidity)